In [2]:
import pandas as pd

In [47]:
#主签单
df = pd.read_excel(r"C:\Users\U0014748\Desktop\2024年数据\3\内销\签单\接单出货1-3月.XLSX", sheet_name = '接单')
#df1 = pd.read_excel("C:\\Users\\U0014748\\Desktop\\2024年数据\\一月数据\\签单（未处理）\\新-综合\\1.2 2024年1月深圳洲明新签订单表—只是保留23年租赁.xlsx", sheet_name = '租赁ZSD110')
#照明
df2 = pd.read_excel(r"C:\Users\U0014748\Desktop\2024年数据\3\内销\签单\2024年1-3月签单明细20240401照明.xlsx", sheet_name = '签单')
#文创
df3 = pd.read_excel(r"C:\Users\U0014748\Desktop\2024年数据\3\内销\签单\2024年文创签单&出货明细（截至3.31）.xlsx", sheet_name = '2024年订单明细')
#KAKC
big_cust = pd.read_excel("C:\\Users\\U0014748\\Desktop\\2024年数据\\一月数据\\签单（未处理）\\KAKC清单汇总20230508.xlsx", sheet_name = 'KAKC清单')

In [48]:
#df1 = df1[['合同号码', '产品线辅助-建高','产品类型(VA03)']]
big_cust = big_cust[['客户名称','分类']]

In [49]:

pattern = '深圳市洲明科技股份有限公司|深圳市海泰装备有限公司|广东洲明节能科技有限公司|成都洲明科技有限公司|海南洲明科技有限公司'


df = df[~df['客户名称'].str.contains(pattern)]


In [50]:
df['审批日期'] = pd.to_datetime(df['审批日期'])

#Extract the month from the '审批日期' column and assign it to the '月份' column
df['月份'] = df['审批日期'].dt.month

In [51]:
df['产品线2'] = None

#Conditionally set values in '产品线2' based on '产品类型(VA03)'
df.loc[df['产品类型(VA03)'] == '文创产品', '产品线2'] = '文创'


df.loc[df['产品线'] == '蓝普国内', '产品线2'] = '蓝普国内'

In [52]:
#蓝普国内：依据【产品线】字段，为“蓝普国内”的归为蓝普国内。然后依据大客户清单检索【客户名称】，区分哪些客户为：【渠道KC】
有蓝普 = df[df['产品线']=='蓝普国内']
无蓝普 = df[df['产品线']!='蓝普国内']


有蓝普.loc[有蓝普['产品线'] == '蓝普国内', '产品线2'] = '蓝普国内'
蓝普kc = pd.merge(有蓝普,big_cust, left_on='客户名称', right_on='客户名称', how='left')

蓝普kc.loc[蓝普kc['分类'].notna(), '产品线2'] = 蓝普kc['分类']
#蓝普kc = 蓝普kc.drop('分类',axis=1)

df = pd.concat([蓝普kc, 无蓝普])

In [53]:
df.loc[df['产品类型(VA03)'] == '文创产品', '产品线2'] = '文创'
df.loc[df['产品线'] == '洲明分销', '产品线2'] = '洲明分销'
df.loc[df['产品线'] == '洲明分销', '产品线2'] = '洲明分销'
# Merge df with df1 to match '合同号' and bring '产品线辅助-建高' into df
df.loc[df['产品类型(VA03)'].str.contains('租赁', na=False), '产品线2'] = '租赁'

In [54]:
df['产品线2'] = df['产品线2'].fillna('主品牌')

In [55]:
df['产品线2'].value_counts()

产品线2
主品牌     712
蓝普国内    467
洲明分销    460
租赁       59
文创        2
Name: count, dtype: int64

In [56]:
df.loc[df['产品线2'].isin(['蓝普国内', '渠道KC', 'KC']), '经营数据1'] = '蓝普国内'
df.loc[df['产品线2'] == '洲明分销', '经营数据1'] = '洲明分销'
df.loc[df['产品线2'] == '主品牌', '经营数据1'] = '主品牌'
df.loc[df['产品线2'].isin(['文创产品', '文创']), '经营数据1'] = '文创产品'
df.loc[df['产品线2'] == '租赁', '经营数据1'] =  '主品牌'


In [57]:
df['经营数据2'] = None

In [58]:
def fill_conditionally(i):
    if i == '主品牌':
        return '主品牌'
    else:
        return ''

# Apply the function across the DataFrame rows
df['经营数据2'] = df['经营数据1'].apply(fill_conditionally)

In [59]:
df['经营数据2'].value_counts()

经营数据2
       929
主品牌    771
Name: count, dtype: int64

In [60]:
df.loc[df['产品类型(VA03)'] == 'UTV', '经营数据2'] = 'UTV'

In [61]:
df['经营数据2'].value_counts()

经营数据2
       929
主品牌    653
UTV    118
Name: count, dtype: int64

In [62]:
df.loc[df['产品线2'] == '租赁', '经营数据2'] = '租赁'

In [63]:
df['经营数据2'].value_counts()

经营数据2
       929
主品牌    594
UTV    118
租赁      59
Name: count, dtype: int64

In [64]:
# Define the mapping based on the provided image
category_mapping = {
    "安防": "安防",
    "大应急": "其他",
    "分销囤货": "其他",
    "公检法司": "其他",
    "交通": "交通",
    "金融": "金融",
    "军队": "UP",
    "民生": "民生",
    "能源": "能源",
    "其他": "其他",
    "渠道囤货": "其他",
    "商显": "其他",
    "数字展陈": "数字展陈",
    "体育": "体育",
    "新媒体": "其他",
    "新商业": "新商业",
    "智慧城市": "其他",
    "智慧党建": "其他",
    "智慧会议": "其他",
    "智慧教育": "其他",
    "租赁": "其他",
    "文娱": "文娱",
    "教育": "教育",
    "政法行业": "其他",
    "智慧光显": "其他"
}

# Process the list to change each category based on the mapping
df['行业类型-经营数据'] = [category_mapping.get(category) for category in df['行业类型']]

df['行业类型-经营数据']=df['行业类型-经营数据'].fillna('其他')


In [65]:
df['签单表']= '主签单表'
df['销售类型']='内销'

In [66]:
df.columns

Index(['订单类型', '销售订单号', '创建日期', '审批日期', '客户名称', '合同号', '产品型号', '产品类型(VA03)',
       '产品线', '洲明工程/洲明分销', '业绩金额', '业绩数量', '业绩面积', '质保期', '销售部门描述', '省份', '国家',
       '雇员部门名称', '业务员', '业务助理', '项目名称', '行业类型', '行业类别', '灯的型号', '灯的品牌', '业绩比例',
       '业绩分配说明', '订单数量', '产品类别(VA03)', '月份', '产品线2', '分类', '经营数据1', '经营数据2',
       '行业类型-经营数据', '签单表', '销售类型'],
      dtype='object')

df = df.set_index('签单表')
df.to_excel('主签单202403.xlsx')

# 照明

In [67]:
df2['签单表']= '照明'
df2['销售类型']= '内销'
df2['产品线2']= '照明'

In [68]:
pattern = '深圳市洲明科技股份有限公司|深圳市海泰装备有限公司|广东洲明节能科技有限公司|成都洲明科技有限公司|海南洲明科技有限公司'

In [69]:
df2 = df2[~df2['公司名称'].str.contains(pattern, na=False)]

In [70]:
df2['日期'] = pd.to_datetime(df2['日期'])

# Extract the month from the '审批日期' column and assign it to the '月份' column
df2['月份'] = df2['日期'].dt.month

In [71]:
import numpy as np

df2['区域'] = df2['项目的归属'].apply(lambda x: x.strip('区') if isinstance(x, str) else np.nan)


df2.to_excel('照明202403.xlsx')

# 文创

In [72]:
df3['签单表']= '文创'
df3['销售类型']='内销'
df3['产品线2']= '文创'

In [73]:
df3 = df3[~df3['客户名称'].str.contains(pattern, na=False)]
df3 = df3[~df3['部门'].str.contains('外贸', na=False)]

df3.to_excel('文创202403.xlsx')

# 海外签单表

In [74]:
海外签单 = pd.read_excel(r"C:\Users\U0014748\Desktop\2024年数据\3\20240301 签单.XLSX", sheet_name = '台账')
海外签单['含税总价（本位币)'] = 海外签单['含税总价（本位币)'] / 10000
海外签单['销售类型']='外销'
海外签单['签单表']='海外签单表'

In [75]:
海外签单 = 海外签单.rename(columns = {
    'SD 凭证':'销售订单号',
    '审批时间':'审批日期',
    '审批月':'月份',
    '采购订单号码':'合同号',
    '含税总价（本位币)':'业绩金额（万）',
    '销售部门':'销售部门描述',
    '销售组':'国家',
    '名':'业务员'
})

In [33]:
海外签单.columns

Index(['SOrg.', '销售订单号', '创建日期', '审批日期', '月份', '客户名称', '合同号', '美元金额',
       '业绩金额（万）', '凭证货币', '业务员', '业务助理', '销售部门描述', '国家', '行业类型', '产品名称', '点间距',
       '总面积', '项目名称', '售达方', '业绩分配说明', '挪用信息', '请求交货日期', '行业类别', '雇员部门名称',
       '销售类型', '签单表'],
      dtype='object')

海外签单.to_excel('海外签单.xlsx')

# 合并

In [76]:
df['业绩金额'] = df['业绩金额'] / 10000
df2['合同金额'] = df2['合同金额'] / 10000
# Rename the '业绩金额' column to '业绩金额（万）'
df = df.rename(columns={'业绩金额': '业绩金额（万）'})

In [77]:
df2.drop('序号', axis=1, inplace=True)

In [78]:
# 照明
df2 = df2.rename(columns={
    '合同金额': '业绩金额（万）',
    '区域': '销售部门描述',
    '项目地': '省份',
    '业务员': '业务员',  # This line and the next one can be omitted as they don't change anything.
    '月份': '月份',
    '公司名称': '客户名称',
    '日期': '审批日期'
})



#文创
df3 = df3.rename(columns={
  # This is duplicated, so you only need to mention it once.
    '日期': '审批日期',
    '客户名称': '客户名称',  # This line can be omitted as it doesn't change anything.
    '订单号': '合同号',
    '产品规格': '产品型号',
    '产品系列': '产品类型(VA03)',
    '折算RMB金额\n（万元）': '业绩金额（万）',
    '业绩归属区域': '销售部门描述',  # This is duplicated, so you only need to mention it once.
    '项目地\n（业绩归属省份）': '省份',
    '部门': '雇员部门名称',
    '业务员': '业务员',  # This line can be omitted as it doesn't change anything.
    '月份': '月份',
    '合同编号/项目名称': '项目名称'
})


In [79]:
海外签单.columns

Index(['SOrg.', '销售订单号', '创建日期', '审批日期', '月份', '客户名称', '合同号', '美元金额',
       '业绩金额（万）', '凭证货币', '业务员', '业务助理', '销售部门描述', '国家', '行业类型', '产品名称', '点间距',
       '总面积', '项目名称', '售达方', '业绩分配说明', '挪用信息', '请求交货日期', '行业类别', '雇员部门名称',
       '销售类型', '签单表'],
      dtype='object')

In [80]:
df_merged = pd.concat([df, df2, df3, 海外签单], ignore_index=True, sort=False)


df_merged = df_merged.set_index('签单表')
df_merged.to_excel('签单合并表2024第一季度.xlsx')

In [81]:
df_merged.columns

Index(['订单类型', '销售订单号', '创建日期', '审批日期', '客户名称', '合同号', '产品型号', '产品类型(VA03)',
       '产品线', '洲明工程/洲明分销', '业绩金额（万）', '业绩数量', '业绩面积', '质保期', '销售部门描述', '省份',
       '国家', '雇员部门名称', '业务员', '业务助理', '项目名称', '行业类型', '行业类别', '灯的型号', '灯的品牌',
       '业绩比例', '业绩分配说明', '订单数量', '产品类别(VA03)', '月份', '产品线2', '分类', '经营数据1',
       '经营数据2', '行业类型-经营数据', '签单表', '销售类型', '合同模式', '签约公司', '项目的归属', '洲明采购单号',
       'SOrg.', '美元金额', '凭证货币', '产品名称', '点间距', '总面积', '售达方', '挪用信息', '请求交货日期'],
      dtype='object')

In [82]:
df_merged['审批日期'] = pd.to_datetime(df_merged['审批日期'])

#Extract the month from the '审批日期' column and assign it to the '月份' column
df_merged['年份'] = df_merged['审批日期'].dt.year

In [84]:
pd.DataFrame(df_merged.groupby(['签单表', '月份'])['业绩金额（万）'].sum())

业绩金额（万）
签单表   月份              
主签单表  1   18845.287848
      2    5753.766632
      3   31884.107740
文创    1    1819.692000
      2     548.269200
      3       4.140000
海外签单表 1   16644.570763
      2   13853.759468
照明    1    2839.981988
      2     390.725412
      3    1826.457569

In [85]:
df_merged.to_excel('签单2024一季度（除海外子公司）.xlsx')

In [ ]:
pd.DataFrame(df_merged.groupby([]))

# 测试使用